In [1]:
import lib.core as C

In [2]:
import smv.functions as f

In [2]:
x=C.readPubMedXml('/data/pdda_raw/17_pubmed/2018baseline/pubmed18n0620.xml.gz', 'lib/pubmed_mini_schema.json', sqlContext)

In [9]:
def m1(df):
    u=F.udf(
            lambda aa: '|'.join([e['_VALUE'] for s in aa for e in s if e is not None]) if isinstance(aa, (list, list)) else None
        )
    return df.select(u(F.col('KeywordList.Keyword')).alias('k'), F.col('PMID._VALUE').alias('ID'), 'Article.ArticleTitle')

In [10]:
o1=m1(x).cache()

In [11]:
now();c1=o1.count();now();c1

09:58:36
09:59:44


30000

In [8]:
o1.where(o1.k.isNotNull()).peek()

k:String = Pregnancy outcome|case-control.|systemic lupus erythematosus (SLE)


In [44]:
def m2(df):
    df1 = df.select(F.col('PMID._VALUE').alias('ID'), F.explode('KeywordList.Keyword').alias('k1'))
    df2 = df1.select('ID', F.explode('k1._VALUE').alias('k2'))
    df3 = df2.groupBy('ID').agg(F.collect_list('k2').alias('keys')).select('ID', f.smvArrayCat('|', F.col('keys')).alias('kstr'))
    df4 = df.select(F.col('PMID._VALUE').alias('ID'), 'Article.ArticleTitle')
    res = df4.smvJoinByKey(df3, ['ID'], 'leftouter')
    return res

In [29]:
df1 = x.select(F.col('PMID._VALUE').alias('ID'), F.explode('KeywordList.Keyword').alias('k1'))
df2 = df1.select('ID', F.explode('k1._VALUE').alias('k2'))

In [36]:
df3 = df2.groupBy('ID').agg(F.collect_list('k2').alias('keys')).select('ID', f.smvArrayCat('|', F.col('keys')).alias('kstr'))

In [37]:
df3.peek()

ID:Long     = 19165351
kstr:String = TC RNA amplification|aRNA|brain extract|column filtration|in vitro transcription|microarray


In [45]:
now();c2=m2(x).count();now();c2

10:21:38
10:22:37


30000

In [41]:
x.select(F.col('PMID._VALUE').cast('string').alias('ID')).smvEdd('ID')

ID                   Non-Null Count         30000
ID                   Null Count             0
ID                   Min Length             8
ID                   Max Length             8
ID                   Approx Distinct Count  30043


# Check for larger files

In [111]:
x=C.readPubMedXml('/data/pdda_raw/17_pubmed/2018baseline/pubmed18n062*.xml.gz', 'lib/pubmed_mini_schema.json', sqlContext)

In [47]:
now();c1=m1(x).count();now();c1

10:23:53
10:28:13


300000

In [48]:
now();c2=m2(x).count();now();c2

10:28:46
10:33:54


300000

In [100]:
from pyspark.sql.types import *
def m3(df):
    dt=df.select('KeywordList.Keyword').schema.fields[0].dataType
    df0 = df.select(F.col('PMID._VALUE').alias('ID'), 'Article.ArticleTitle', F.coalesce(
        F.col('KeywordList.Keyword'), F.array(F.array(F.lit(None).cast(dt.elementType.elementType)).cast(dt.elementType)).cast(dt)).alias('k0'))
    df1 = df0.select('ID', 'ArticleTitle', F.explode('k0').alias('k1'))
    df2 = df1.select('ID', 'ArticleTitle', F.explode('k1._VALUE').alias('k2'))
    df3 = df2.groupBy('ID').agg(F.first('ArticleTitle').alias('ArticleTitle'), F.collect_list('k2').alias('keys')).select('ID', f.smvArrayCat('|', F.col('keys')).alias('kstr'))
    return df3

In [106]:
def explode_keep_null(df, c):
    dt = df.select(c).schema.fields[0].dataType
    return df.withColumn(c, F.explode(F.coalesce(
            F.col(c), F.array(F.lit(None).cast(dt.elementType)).cast(dt))))
def m32(df):
    df0 = df.select(F.col('PMID._VALUE').alias('ID'), 'Article.ArticleTitle', 'KeywordList.Keyword')
    df1 = explode_keep_null(df0, 'Keyword')
    df2 = explode_keep_null(df1, 'Keyword').select('ID', 'ArticleTitle', F.col('Keyword._VALUE').alias('k2'))
    df3 = df2.groupBy('ID').agg(F.first('ArticleTitle').alias('ArticleTitle'), F.collect_list('k2').alias('keys')).select('ID', f.smvArrayCat('|', F.col('keys')).alias('kstr'))
    return df3

In [115]:
def explode_keep_null(df, c):
    dt = df.select(c).schema.fields[0].dataType
    return df.withColumn(c, F.explode(F.coalesce(
            F.col(c), F.array(F.lit(None).cast(dt.elementType)).cast(dt))))

def flatten_array_of_array(df, keys, c, elem):
    df1 = explode_keep_null(df, c)
    df2 = explode_keep_null(df1, c).withColumn(c, F.col(c)[elem])
    kept = [i for i in df.columns if i not in (keys + [c])]
    res = df2.groupBy(*keys).agg(*([F.first(i).alias(i) for i in kept] + [F.collect_list(c).alias(c)]))
    return res

def m33(df):
    df0 = df.select(F.col('PMID._VALUE').alias('ID'), 'Article.ArticleTitle', 'KeywordList.Keyword')
    df1 = flatten_array_of_array(df0, ['ID'], 'Keyword', '_VALUE')
    res = df1.select('ID', 'ArticleTitle', f.smvArrayCat('|', F.col('Keyword')).alias('k'))
    return res

In [93]:
x.select('KeywordList.Keyword').schema.fields[0].dataType

ArrayType(ArrayType(StructType(List(StructField(_MajorTopicYN,StringType,true),StructField(_VALUE,StringType,true))),true),true)

In [101]:
now();c3=m3(x).count();now();c3

12:23:28
12:26:29


300000

In [107]:
now();c3=m32(x).cache().count();now();c3

12:54:05
12:57:08


300000

In [116]:
now();c3=m33(x).cache().count();now();c3

13:12:46
13:16:15


300000

# Even larger

In [108]:
x=C.readPubMedXml('/data/pdda_raw/17_pubmed/2018baseline/pubmed18n06[2345]*.xml.gz', 'lib/pubmed_mini_schema.json', sqlContext)

In [109]:
now();c3=m32(x).count();now();c3

12:57:27
13:08:07


1200000

In [53]:
now();c1=m1(x).count();now();c1

10:50:46
11:05:28


1200000

In [54]:
x.select(F.col('PMID._VALUE').cast('string').alias('ID')).smvEdd('ID')

ID                   Non-Null Count         1200000
ID                   Null Count             0
ID                   Min Length             8
ID                   Max Length             8
ID                   Approx Distinct Count  1188800


# Check for Scala version

In [3]:
x1=C.readPubMedXml('/data/pdda_raw/17_pubmed/2018baseline/pubmed18n062*.xml.gz', 'lib/pubmed_mini_schema.json', sqlContext)

In [10]:
def m4(df):
    df0 = df.select(F.col('PMID._VALUE').alias('ID'), 'Article.ArticleTitle', 'KeywordList.Keyword')
    df1 = df0.withColumn('Keyword', F.col('Keyword').smvArrayFlatten(df0))
    res = df1.select('ID', 'ArticleTitle', f.smvArrayCat('|', F.col('Keyword._VALUE')).alias('k'))
    return res

In [11]:
now();r4=m4(x1).cache();c4=r4.count();now();c4

19:38:02
19:41:18


300000

In [12]:
r4.where(F.col('k').isNotNull()).peek()

ID:Long             = 19443873
ArticleTitle:String = An assessment of felt needs and expectations of people living with HIV/AIDS seeking treatment at NGOs in Delhi, India.
k:String            = communicable diseases|health care services|health services evaluation|medicolegal issues|resources


In [7]:
x2=C.readPubMedXml('/data/pdda_raw/17_pubmed/2018baseline/pubmed18n06[2345]*.xml.gz', 'lib/pubmed_mini_schema.json', sqlContext)

In [9]:
now();r5=m4(x2).cache();c5=r5.count();now();c5

19:09:40
19:20:27


1200000